### Import packages

Import of the packages that will be needed for the project.  This includes packages for data manipulation, sklearn modules and custom functions.

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, FunctionTransformer

from Project_Functions import offensive_contribution, trailing_stats_mean, tier_maker, get_tiers, get_touchdowns, stats_for_trailing

In [2]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = 'ignore')

### Import Data

Let's import the dataframe that we will be using for modelling

In [3]:
data = pd.read_csv('Data/weekly_data.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0


In [4]:
data.isna().sum()

Unnamed: 0      0
Unnamed: 0.1    0
GameKey         0
PlayerID        0
Season          0
               ..
HomePoints      0
HomeYards       0
AwayTeam        0
AwayPoints      0
AwayYards       0
Length: 65, dtype: int64

Before the train test split, we have to calculate the trailing average fantasy points for each observation, as we cannot incorporate this step into the pipeline without causing data leakage.

In [5]:
def trailing_stats(df):
    
    """
    Function to create a new column with a trailing aggregate mean
    as a new feature for prediction.
    
    Inputs:
        - df: The dataframe on which the function will be applied
        - Column: The column on which to apply the function
        - Window: The number of past values to consider when apply the function

        
    Output:
        - An aggregate value
        
    """
    #Access the column names in stats_for_trailing
    global stats_for_trailing
    
    # Get all unique players in the DataFrame
    players = df['Name'].unique().tolist()
    
    # Define a DataFrame to hold our values
    df_out = pd.DataFrame()
    # Loop through the unique players
    for player in players:
        # Create a temporary dataframe for each player
        temp_df = df[df['Name'] == player]
        # Calculate the n game trailing average for all players.  Set closed parameter to 'left'
        # so that the current value for fantasy points is not included in the calculation.
        # Backfill the two resulting NaN values
        for column in stats_for_trailing:
            temp_df[f'TA{column}'] = temp_df[column].rolling(window = 5, 
                                                              closed = 'left').mean().fillna(method = 'bfill')
        # Append the temporary dataframe to the output
        df_out = df_out.append(temp_df)
    # Return a dataframe with the values sorted by the original index
    return df_out.sort_index()
    

In [6]:
stats_for_trailing = ['FantasyPointsPPR']

In [7]:
# Prepare the trailing average fantasy points column

data = trailing_stats(data)
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards,TAFantasyPointsPPR
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,20.088
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,28.600
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,25.204
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0,18.240
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0,22.420


### Train Test Split

In [8]:
# Separate data from the target
y = data['FantasyPointsPPR']
data.drop(columns = ['FantasyPointsPPR'],
            inplace = True)
X = data

In [9]:
# Execute the train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.25,
                                                   random_state = 13,
                                                   shuffle = False)

### Feature Engineering

The main features that we will be engineering to predict a player's fantasy output will be the 5-game trailing average of various statistics as well as the binning of players into their respective tiers based on recent performance.

In [10]:
# Define the columns for which we want a 5 game trailing average.

stats_for_trailing = ['TotalTouchdowns','RushingYards','PassingInterceptions','PassingTouchdowns','PassingRating','PassingYards',
                      'PassingCompletionPercentage','RushingYards', 'RushingTouchdowns', 'RushingLong',
                      'Receptions','ReceivingYards','ReceivingTargets', 'ReceivingTouchdowns', 'ExtraPointsMade', 'FieldGoalsMade',
                      'Fumbles','FumblesLost']

trailing_stats = []
for col in stats_for_trailing:
    trailing_stats.append('TA' + col)
trailing_stats

['TATotalTouchdowns',
 'TARushingYards',
 'TAPassingInterceptions',
 'TAPassingTouchdowns',
 'TAPassingRating',
 'TAPassingYards',
 'TAPassingCompletionPercentage',
 'TARushingYards',
 'TARushingTouchdowns',
 'TARushingLong',
 'TAReceptions',
 'TAReceivingYards',
 'TAReceivingTargets',
 'TAReceivingTouchdowns',
 'TAExtraPointsMade',
 'TAFieldGoalsMade',
 'TAFumbles',
 'TAFumblesLost']

In [11]:
# Instantiate the function transformers for the feature engineering pipeline

touchdown_transformer = FunctionTransformer(get_touchdowns) # Get total touchdowns per player
trailing_transformer = FunctionTransformer(trailing_stats_mean) # Get the 5 game trailing averages of appropriate statistics
tier_transformer = FunctionTransformer(get_tiers) # Bin players into the appropriate tiers based on recent performance


# Instantiate the pipeline for the necessary transformations

engineering = Pipeline([('touchdown', touchdown_transformer),
                       ('trailing', trailing_transformer),
                       ('tier', tier_transformer)])

<br>

### Preprocessing

As shown above, the bulk of the null values fall into one of two categories.  They are either:
* In the InjuryStatus column
    * Here we can impute a value of healthy, as the only values in the injury column are 
* In the TA (trailing average) columns we created
    * No player with a null value played more than 5 games, therefore we cannot calculate the trailing average for them.  We will impute a default value of 0 for these columns, as they represent players who likely did not have much impact.  If they had an impact, they likely would have played in more games.  I will explore imputing the median value as well through a grid search.

In [12]:
# Define the groups of columns for preprocessing steps.

categorical_columns = ['Week',
                       'Team',
                       'Name',
                      'Opponent',
                      'PlayerTier',
                      'InjuryStatus']
numerical_columns = trailing_stats


In [13]:
# Define the preprocessing steps for categorical features
categorical_transform = Pipeline([('impute_cat',SimpleImputer(strategy = 'constant',
                                                              fill_value = 'Healthy')),
                                 ('one_hot_encoder', OneHotEncoder(handle_unknown = 'ignore'))])

# Define the preprocessing steps for numerical features
numerical_transform = Pipeline([('impute_num', SimpleImputer(strategy = 'constant',
                                                             fill_value = 0)),
                               ('scaler', StandardScaler())])

# Instantiate the column transformer object for the preprocessing steps
preprocessing = ColumnTransformer([('num', numerical_transform, numerical_columns),
                                  ('cat', categorical_transform, categorical_columns)])

In [14]:
# Instantiate a pipeline with a linear regression model as a baseline

pipeline = Pipeline([('engineering', engineering),
                    ('prep', preprocessing),
                    ('model', RandomForestRegressor(verbose = 3))])

In [15]:
# Set param grid values, parameters for grid search 

param_grid = {'model__n_estimators': [100],
             'model__max_depth': [5, 6, 7]}

grid_search = GridSearchCV(pipeline, 
                          param_grid = param_grid,
                          cv = 3,
                          verbose = 3,
                          n_jobs = -2)

In [16]:
# Fit the grid search to X_train and y_train

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/Users/bernardpriest-blais/Desktop/LHL_Final_Project/Project_Functions.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[f'TA{column}'] = temp_df[column].rolling(window = 5,
/Users/bernardpriest-blais/Desktop/LHL_Final_Project/Project_Functions.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[f'TA{column}'] = temp_df[column].rolling(window = 5,
/Users/bernardpriest-blais/Desktop/LHL_Final_Project/Project_Functions.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy o

building tree 1 of 100
building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.8s finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('engineering',
                                        Pipeline(steps=[('touchdown',
                                                         FunctionTransformer(func=<function get_touchdowns at 0x7f806f119c10>)),
                                                        ('trailing',
                                                         FunctionTransformer(func=<function trailing_stats_mean at 0x7f806f119700>)),
                                                        ('tier',
                                                         FunctionTransformer(func=<function get_tiers at 0x7f806f119b80>))])),
                                       ('prep',
                                        ColumnTransformer(transform...
                                                                        ('cat',
                                                                         Pipeline(steps=[('impute_cat',
                             

In [17]:
grid_search.best_score_

0.5031805724612254

In [18]:
preds = grid_search.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [19]:
mean_squared_error(y_test, preds)

28.72127263538214

In [20]:
mean_absolute_error(y_test, preds)

3.578060967511565

Let's dig into the predictions a little bit

In [21]:
y_test

23653    16.00
23654    15.84
23655    24.80
23656    17.70
23657    21.50
         ...  
31533    -0.20
31534    -0.30
31535     0.60
31536    -0.30
31537    -1.44
Name: FantasyPointsPPR, Length: 7885, dtype: float64

In [22]:
df_preds = pd.DataFrame(y_test)

In [23]:
df_preds['Predicted'] = preds

In [28]:
df_preds.sort_values(by = 'Predicted', ascending = False).head(50)

,FantasyPointsPPR,Predicted
30773,7.30,22.573660
27081,32.26,22.505487
26584,18.00,22.471603
30111,24.40,22.251645
30687,11.20,21.813238
25360,21.32,21.811748
24752,29.22,21.811748
27624,19.28,21.697318
29198,22.98,21.584801
24160,34.92,21.507779
